# Plantilla para la Tarea online BDA03

# Nombre del alumno: Jacobo Rodríguez Esmorís

Realiza las tareas que se plantean en cada ejercicio. En algunas tareas deberás completar las celdas que están incompletas en otras añadir nuevas celdas. Se trata de que implementes una serie de consultas con HQL (Hive) y Pig Latin.

Vamos a seguir utilizando el `dataset` de retrasos en vuelos en EEUU de la guía práctica. A modo de recordatorio, en el siguiente apartado, repetimos la explicación del significado de los campos.

# Dataset de retrasos en vuelos

Vamos a usar [este](https://www.kaggle.com/datasets/tylerx/flights-and-airports-data) de Kaggle
para aprender a usar tanto Hive como Pig. Kaggle es un sitio muy popular en ciencia de datos. En este sitio los científicos de datos pueden publicar y compartir sus trabajos. Además también se pueden proponer concursos en los que los participantes compiten en la construcción del mejor modelo para el problema propuesto.

El `dataset` contiene información sobre retrasos en vuelos en EEUU. Hay dos ficheros de interés: `airports.csv` y `flights.csv`.

El primero tiene información sobre los aeropuertos y consta de los siguientes campos:
   * airport_id: identificador del aeropuerto. Numérico, aunque se utilizará un campo `string` en Hive.
   * city: ciudad del aeropuerto.
   * state: estado del aeropuerto.
   * name: nombre del aeropuerto.
   
El fichero `flights` tiene la siguiente estructura:
   * DayofMonth: día del mes del vuelo.
   * DayOfWeek: día de la semana del vuelo.
   * Carrier: Identificador de la compañía aérea.
   * OriginAirportID: Identificador del aeropuerto de origen.
   * DestAirportID: Identificador del aeropuerto de destino.
   * DepDelay: Minutos de retraso en la salida de un vuelo (puede ser negativo si el vuelo sale antes de lo previsto).
   * ArrDelay: Minutos de retraso en la llegada de un vuelo (puede ser negativo si el vuelo sale antes de lo previsto).

El directorio `notebooks` contiene el `archiv.zip` con los dos ficheros. Para descargarlo de Kaggle hay que estar registrado y se ha incluido para que no tengas que registrarte.

## 1.- Realiza el proceso de preparación que se hizo en la guia práctica:

* Crea las celdas y muestra el resultado de su ejecución de la extracción de los ficheros del `dataset` de vuelos.
* Crea la base de datos de Hive y las tablas `airports` y `flights`. Presta atención a cambiar los comentarios y no simplemente copiar los de la guía.
* Carga las tablas y crea consultas de HQL que muestren 10 aeropuertos y 10 vuelos como se hizo en la guía práctica.
* Crea un `script` en Pig Latin que muestre 10 aeropuertos y 10 vuelos como se hizo en la guía práctica.

In [2]:
! apt update

Get:1 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1171 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]3m
Get:7 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3329 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1275 kB]    
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [32.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3806 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-

In [3]:
! apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  zip
The following NEW packages will be installed:
  unzip
0 upgraded, 1 newly installed, 0 to remove and 207 not upgraded.
Need to get 168 kB of archives.
After this operation, 593 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 unzip amd64 6.0-25ubuntu1.1 [168 kB]
Fetched 168 kB in 1s (259 kB/s)
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package unzip.
(Reading database ... 43749 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-25ubuntu1.1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking unzip (6.0-25ubuntu1.1) ...
7Progress: [ 40%] [#######################.......................

In [4]:
! unzip -j -o archive.zip airports.csv flights.csv

Archive:  archive.zip
  inflating: airports.csv            
  inflating: flights.csv             


In [5]:
! wc -l airports.csv && head airports.csv

366 airports.csv












In [7]:
! wc -l flights.csv && head flights.csv

2702219 flights.csv












In [10]:
! hdfs dfs -mkdir -p /user/root/flights/
! hdfs dfs -put -f ./airports.csv /user/root/flights/
! hdfs dfs -put -f ./flights.csv /user/root/flights/
! hdfs dfs -ls /user/root/flights/

Found 2 items
-rw-r--r--   3 root supergroup      16308 2024-01-29 12:08 /user/root/flights/airports.csv
-rw-r--r--   3 root supergroup   72088113 2024-01-29 12:08 /user/root/flights/flights.csv


## HIVE

In [20]:
! beeline -u "jdbc:hive2://localhost:10000" -e "SHOW DATABASES"

Connecting to jdbc:hive2://localhost:10000
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240129121703_e3045057-84ce-4532-8786-d8395096b101): SHOW DATABASES
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:database_name, type:string, comment:from deserializer)], properties:null)
INFO  : Completed compiling command(queryId=root_20240129121703_e3045057-84ce-4532-8786-d8395096b101); Time taken: 0.921 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20240129121703_e3045057-84ce-4532-8786-d8395096b101): SHOW DATABASES
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20240129121703_e3045057-84ce-4532-8786-d8395096b101); Ti

In [21]:
! beeline -u "jdbc:hive2://localhost:10000/" -e "\
CREATE DATABASE IF NOT EXISTS bda03 \
COMMENT 'Base de datos de la unidad BDA03' \
WITH DBPROPERTIES ('Creada por' = 'Jacobo Rodríguez', 'Fecha' = '29/01/2024');"

Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240129122052_e2f14897-bde7-41aa-b3d7-4f9c73024191): CREATE DATABASE IF NOT EXISTS bda03  COMMENT 'Base de datos de la unidad BDA03'  WITH DBPROPERTIES ('Creada por' = 'Jacobo Rodr?guez', 'Fecha' = '29/01/2024')
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20240129122052_e2f14897-bde7-41aa-b3d7-4f9c73024191); Time taken: 0.043 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20240129122052_e2f14897-bde7-41aa-b3d7-4f9c73024191): CREATE DATABASE IF NOT EXISTS bda03  COMMENT 'Base de datos de la unidad BDA03'  WITH DBPROPERT

In [24]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
DROP TABLE IF EXISTS airports; \
CREATE EXTERNAL TABLE IF NOT EXISTS airports (airportid STRING, city STRING, state STRING, airportname STRING) \
COMMENT 'USA Airports' \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' \
TBLPROPERTIES ('Autor' = 'Jacobo Rodríguez', 'Fecha' = '29/01/2024', 'skip.header.line.count'='1');"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240129122942_8c1c3745-ee31-437d-849c-3ee5c5225c6d): DROP TABLE IF EXISTS airports
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20240129122942_8c1c3745-ee31-437d-849c-3ee5c5225c6d); Time taken: 0.098 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20240129122942_8c1c3745-ee31-437d-849c-3ee5c5225c6d): DROP TABLE IF EXISTS airports
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20240129122942_8c1c3745-ee31-437d-849c-3ee5c5225c6d); Time taken: 0.031 seconds
INFO  : O

In [25]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
SELECT * FROM airports;"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240129123049_2506c397-541f-42ba-8eb3-ad889409e3a7): SELECT * FROM airports
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:airports.airportid, type:string, comment:null), FieldSchema(name:airports.city, type:string, comment:null), FieldSchema(name:airports.state, type:string, comment:null), FieldSchema(name:airports.airportname, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20240129123049_2506c397-541f-42ba-8eb3-ad889409e3a7); Time taken: 1.199 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20240129123049_2506c397-541f-42

In [26]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e " \
DROP TABLE IF EXISTS flights; \
CREATE EXTERNAL TABLE IF NOT EXISTS flights (dayofmonth TINYINT, dayofweek TINYINT, carrier STRING, \
    depairportid STRING, arrairportid STRING, depdelay SMALLINT, arrdelay SMALLINT) \
COMMENT 'Flights' \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' \
TBLPROPERTIES ('Autor' = 'Jacobo Rodríguez', 'Fecha' = '29/01/2024', 'skip.header.line.count'='1');"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240129124713_86eb1b6c-592d-4b01-b6ae-668c98fafcc2): DROP TABLE IF EXISTS flights
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20240129124713_86eb1b6c-592d-4b01-b6ae-668c98fafcc2); Time taken: 0.014 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20240129124713_86eb1b6c-592d-4b01-b6ae-668c98fafcc2): DROP TABLE IF EXISTS flights
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20240129124713_86eb1b6c-592d-4b01-b6ae-668c98fafcc2); Time taken: 0.017 seconds
INFO  : OK


In [27]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
SELECT * FROM flights;"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240129124735_442ff850-d3b9-4579-9e99-ee42f6f798bf): SELECT * FROM flights
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:flights.dayofmonth, type:tinyint, comment:null), FieldSchema(name:flights.dayofweek, type:tinyint, comment:null), FieldSchema(name:flights.carrier, type:string, comment:null), FieldSchema(name:flights.depairportid, type:string, comment:null), FieldSchema(name:flights.arrairportid, type:string, comment:null), FieldSchema(name:flights.depdelay, type:smallint, comment:null), FieldSchema(name:flights.arrdelay, type:smallint, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_2024012912473

In [28]:
! hdfs dfs -chmod 777 /user/root/flights/

In [29]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e " \
LOAD DATA INPATH '/user/root/flights/airports.csv' OVERWRITE INTO TABLE airports;"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240129125113_8123de60-3004-400d-9274-d8fe53fca79c): LOAD DATA INPATH '/user/root/flights/airports.csv' OVERWRITE INTO TABLE airports
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20240129125113_8123de60-3004-400d-9274-d8fe53fca79c); Time taken: 0.048 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20240129125113_8123de60-3004-400d-9274-d8fe53fca79c): LOAD DATA INPATH '/user/root/flights/airports.csv' OVERWRITE INTO TABLE airports
INFO  : Starting task [Stage-0:MOVE] in serial mode
INFO  : Loading data to table bda03

In [30]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e " \
LOAD DATA INPATH '/user/root/flights/flights.csv' OVERWRITE INTO TABLE flights;"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240129125156_6f0dac45-7944-4ccd-8d8c-bfef41a90779): LOAD DATA INPATH '/user/root/flights/flights.csv' OVERWRITE INTO TABLE flights
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20240129125156_6f0dac45-7944-4ccd-8d8c-bfef41a90779); Time taken: 0.031 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20240129125156_6f0dac45-7944-4ccd-8d8c-bfef41a90779): LOAD DATA INPATH '/user/root/flights/flights.csv' OVERWRITE INTO TABLE flights
INFO  : Starting task [Stage-0:MOVE] in serial mode
INFO  : Loading data to table bda03.fli

In [31]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e " \
SELECT * FROM airports LIMIT 10"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240129125236_e8e2c449-4f43-4b79-ac50-881a5ea55fd6): SELECT * FROM airports LIMIT 10
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:airports.airportid, type:string, comment:null), FieldSchema(name:airports.city, type:string, comment:null), FieldSchema(name:airports.state, type:string, comment:null), FieldSchema(name:airports.airportname, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20240129125236_e8e2c449-4f43-4b79-ac50-881a5ea55fd6); Time taken: 0.13 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20240129125236_e8e2c449

In [32]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e " \
SELECT * FROM flights LIMIT 10"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240129125505_1271ded6-3936-4531-be34-ea2b588dafc9): SELECT * FROM flights LIMIT 10
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:flights.dayofmonth, type:tinyint, comment:null), FieldSchema(name:flights.dayofweek, type:tinyint, comment:null), FieldSchema(name:flights.carrier, type:string, comment:null), FieldSchema(name:flights.depairportid, type:string, comment:null), FieldSchema(name:flights.arrairportid, type:string, comment:null), FieldSchema(name:flights.depdelay, type:smallint, comment:null), FieldSchema(name:flights.arrdelay, type:smallint, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_2024

In [40]:
%%writefile flights.pig

-- registro de la librería PiggyBank
REGISTER piggybank.jar

-- Lectura CSV

AIRPORTS = LOAD '$airports_file' USING
    org.apache.pig.piggybank.storage.CSVExcelStorage(',','NO_MULTILINE','UNIX','SKIP_INPUT_HEADER')
    AS (airportid:chararray, city:chararray, state:chararray, airportname:chararray);

FLIGHTS = LOAD '$flights_file' USING
    org.apache.pig.piggybank.storage.CSVExcelStorage(',','NO_MULTILINE','UNIX','SKIP_INPUT_HEADER')
    AS (dayofmonth:int, dayofweek:int, carrier:chararray,
           depairport:chararray, arrairportid:chararray, depdelay:int, arrdelay:int);

-- Nos quedamos con 10 aeropuertos
AIRPORTS_10 = LIMIT AIRPORTS 10;

-- Muestra de 10 aeropuertos
DUMP AIRPORTS_10;

-- Mismo procedimiento con los vuelos
FLIGHTS_10 = LIMIT FLIGHTS 10;
DUMP FLIGHTS_10;

Writing flights.pig


In [41]:
! pig -x local -f flights.pig -param airports_file='airports.csv' -param flights_file='flights.csv' -param output_dir='pig/output/flights'

2024-01-29 14:31:22,518 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2024-01-29 14:31:22,519 INFO pig.ExecTypeProvider: Picked LOCAL as the ExecType
2024-01-29 14:31:22,560 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2024-01-29 14:31:22,560 [main] INFO  org.apache.pig.Main - Logging error messages to: /media/notebooks/pig_1706535082558.log
2024-01-29 14:31:22,573 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2024-01-29 14:31:22,681 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2024-01-29 14:31:22,725 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2024-01-29 14:31:22,726 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2024-01-29 14:

## 2.- Con una consulta de HQL muestra: La cinco compañías que más vuelos retrasados tienen.

* El campo `carrier` contiene la compañía aérea.
* Vamos a considerar que un vuelo llega con retraso cuando el vuelo llega más de 15 minutos tarde (campo `arrdelay` > 15).

Se espera el siguiente resultado:

![solución 2](./img/2.png)

In [36]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e " \
SELECT f.carrier, COUNT(*) as delayed_flights \
FROM flights f \
WHERE f.arrdelay > 15 \
GROUP BY f.carrier \
ORDER BY delayed_flights DESC \
LIMIT 5;"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240129133354_4a068ccf-1382-4807-9a5d-1c7684a7bcdd): SELECT f.carrier, COUNT(*) as delayed_flights  FROM flights f  WHERE f.arrdelay > 15  GROUP BY f.carrier  ORDER BY delayed_flights DESC  LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:f.carrier, type:string, comment:null), FieldSchema(name:delayed_flights, type:bigint, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20240129133354_4a068ccf-1382-4807-9a5d-1c7684a7bcdd); Time taken: 0.137 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20240129133354_4a068ccf-1382-4807-9a5d-1c768

## 3.- Con una consulta de HQL muestra: Las 5 compañías que mejor recuperación de tiempo en vuelo tienen.

* Se considera que se ha recuperado el tiempo de un vuelo cuando habiendo salido con retraso (`depdelay` > 15), llega sin retraso (`arraydelay` <= 15).
* Se trata de que muestres las 5 compañías que han recuperado el tiempo en un mayor porcentaje de vuelos que salieron retrasados.

El resultado esperado es el siguiente:

![solución 3](./img/3.png)

In [39]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e " \
SELECT f.carrier, \
       SUM(CASE WHEN f.depdelay > 15 AND f.arrdelay <= 15 THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS percent_recovered \
FROM flights f \
WHERE f.depdelay > 15 \
GROUP BY carrier \
ORDER BY percent_recovered DESC \
LIMIT 5;"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240129134211_137e6b7e-e80a-4e38-b667-efb4532fea02): SELECT f.carrier,         SUM(CASE WHEN f.depdelay > 15 AND f.arrdelay <= 15 THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS percent_recovered  FROM flights f  WHERE f.depdelay > 15  GROUP BY carrier  ORDER BY percent_recovered DESC  LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:f.carrier, type:string, comment:null), FieldSchema(name:percent_recovered, type:decimal(38,17), comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20240129134211_137e6b7e-e80a-4e38-b667-efb4532fea02); Time taken: 0.124 seconds
INFO  : Concurrency mode is disabled, not creating 

## 4.- Resuelve el ejercicio 2 con Pig Latin

El resultado esperado es:

![solución 4](./img/4.png)

In [47]:
%%writefile delayed_flights.pig

-- registro de la librería PiggyBank
REGISTER piggybank.jar

-- Lectura CSV

AIRPORTS = LOAD '$airports_file' USING
    org.apache.pig.piggybank.storage.CSVExcelStorage(',','NO_MULTILINE','UNIX','SKIP_INPUT_HEADER')
    AS (airportid:chararray, city:chararray, state:chararray, airportname:chararray);

FLIGHTS = LOAD '$flights_file' USING
    org.apache.pig.piggybank.storage.CSVExcelStorage(',','NO_MULTILINE','UNIX','SKIP_INPUT_HEADER')
    AS (dayofmonth:int, dayofweek:int, carrier:chararray,
           depairport:chararray, arrairportid:chararray, depdelay:int, arrdelay:int);

-- Seleccionamos los vuelos retrasados
vuelos_retrasados = FILTER FLIGHTS BY arrdelay > 15;

-- Cuenta de los vuelos retrasados relacionandolos con la compañía
contar_retrasos = GROUP vuelos_retrasados BY carrier;
total_retrasos = FOREACH contar_retrasos GENERATE group AS carrier, COUNT(vuelos_retrasados) AS total_retrasos;

-- Ordenar DESC y limitar a 5 las compañías
top5_retrasos = ORDER total_retrasos BY total_retrasos DESC;
top5_retrasos = LIMIT top5_retrasos 5;

-- DUMP del resultado
DUMP top5_retrasos;

Overwriting delayed_flights.pig


In [48]:
! pig -x local -f delayed_flights.pig -param airports_file='airports.csv' -param flights_file='flights.csv' -param output_dir='pig/output/flights'

2024-01-29 15:53:40,892 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2024-01-29 15:53:40,893 INFO pig.ExecTypeProvider: Picked LOCAL as the ExecType
2024-01-29 15:53:40,928 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2024-01-29 15:53:40,928 [main] INFO  org.apache.pig.Main - Logging error messages to: /media/notebooks/pig_1706540020927.log
2024-01-29 15:53:40,938 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2024-01-29 15:53:41,029 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2024-01-29 15:53:41,071 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2024-01-29 15:53:41,072 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2024-01-29 15:

2024-01-29 15:53:42,276 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - File Output Committer Algorithm version is 2
2024-01-29 15:53:42,276 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
2024-01-29 15:53:42,287 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.Task -  Using ResourceCalculatorProcessTree : [ ]
2024-01-29 15:53:42,291 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Processing split: Number of splits :1
Total Length = 33554432
Input split[0]:
   Length = 33554432
   ClassName: org.apache.hadoop.mapreduce.lib.input.FileSplit
   Locations:

-----------------------

2024-01-29 15:53:42,299 [LocalJobRunner Map Task Executor #0] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.PigRecor

2024-01-29 15:53:50,453 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.LocalJobRunner - 
2024-01-29 15:53:50,454 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Starting flush of map output
2024-01-29 15:53:50,454 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Spilling map output
2024-01-29 15:53:50,454 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - bufstart = 0; bufend = 2384890; bufvoid = 104857600
2024-01-29 15:53:50,454 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - kvstart = 26214396(104857584); kvend = 25260444(101041776); length = 953953/6553600
2024-01-29 15:53:50,552 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Finished spill 0
2024-01-29 15:53:50,553 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.Task - Task:attempt_local833473054_0001_m_000001_0 is done. And is in the proce

2024-01-29 15:53:51,294 [pool-4-thread-1] INFO  org.apache.hadoop.mapred.Merger - Down to the last merge-pass, with 1 segments left of total size: 617 bytes
2024-01-29 15:53:51,294 [pool-4-thread-1] INFO  org.apache.hadoop.mapred.LocalJobRunner - 3 / 3 copied.
2024-01-29 15:53:51,299 [pool-4-thread-1] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - File Output Committer Algorithm version is 2
2024-01-29 15:53:51,299 [pool-4-thread-1] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
2024-01-29 15:53:51,301 [pool-4-thread-1] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.skip.on is deprecated. Instead, use mapreduce.job.skiprecords
2024-01-29 15:53:51,301 [pool-4-thread-1] INFO  org.apache.pig.impl.util.SpillableMemoryManager - Selected heap (PS Old Gen) of size 699400192 to monitor. collectionUsageThreshold = 489580128, us

2024-01-29 15:53:51,601 [LocalJobRunner Map Task Executor #0] INFO  org.apache.pig.impl.util.SpillableMemoryManager - Selected heap (PS Old Gen) of size 699400192 to monitor. collectionUsageThreshold = 489580128, usageThreshold = 489580128
2024-01-29 15:53:51,601 [LocalJobRunner Map Task Executor #0] WARN  org.apache.pig.data.SchemaTupleBackend - SchemaTupleBackend has already been initialized
2024-01-29 15:53:51,602 [LocalJobRunner Map Task Executor #0] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.PigGenericMapReduce$Map - Aliases being processed per job phase (AliasName[line,offset]): M: top5_retrasos[24,16] C:  R: 
2024-01-29 15:53:51,603 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.LocalJobRunner - 
2024-01-29 15:53:51,603 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Starting flush of map output
2024-01-29 15:53:51,603 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Spilli

2024-01-29 15:53:51,774 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - HadoopJobId: job_local1379808152_0002
2024-01-29 15:53:51,774 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - Processing aliases top5_retrasos
2024-01-29 15:53:51,774 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - detailed locations: M: top5_retrasos[24,16] C:  R: 
2024-01-29 15:53:51,775 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - 50% complete
2024-01-29 15:53:51,776 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-01-29 15:53:51,777 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-01-29 15:53:51,778 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system alrea

2024-01-29 15:53:51,904 [pool-12-thread-1] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
2024-01-29 15:53:51,906 [pool-12-thread-1] INFO  org.apache.pig.impl.util.SpillableMemoryManager - Selected heap (PS Old Gen) of size 699400192 to monitor. collectionUsageThreshold = 489580128, usageThreshold = 489580128
2024-01-29 15:53:51,906 [pool-12-thread-1] WARN  org.apache.pig.data.SchemaTupleBackend - SchemaTupleBackend has already been initialized
2024-01-29 15:53:51,907 [pool-12-thread-1] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.PigMapReduce$Reduce - Aliases being processed per job phase (AliasName[line,offset]): M: top5_retrasos[24,16] C:  R: 
2024-01-29 15:53:51,908 [pool-12-thread-1] INFO  org.apache.hadoop.mapred.Task - Task:attempt_local1629033974_0003_r_000000_0 is done. And is in the process of committing
2024-01-29 15:53:51

2024-01-29 15:53:52,164 [pool-15-thread-1] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - File Output Committer Algorithm version is 2
2024-01-29 15:53:52,164 [pool-15-thread-1] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
2024-01-29 15:53:52,165 [pool-15-thread-1] INFO  org.apache.hadoop.mapred.Task -  Using ResourceCalculatorProcessTree : [ ]
2024-01-29 15:53:52,165 [pool-15-thread-1] INFO  org.apache.hadoop.mapred.ReduceTask - Using ShuffleConsumerPlugin: org.apache.hadoop.mapreduce.task.reduce.Shuffle@4f8e16
2024-01-29 15:53:52,165 [pool-15-thread-1] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-01-29 15:53:52,166 [pool-15-thread-1] INFO  org.apache.hadoop.mapreduce.task.reduce.MergeManagerImpl - MergerManager: memoryLimit=652528832, maxSingleShuffleLimit=163132208, 

2024-01-29 15:53:52,364 [main] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2024-01-29 15:53:52,364 [main] INFO  org.apache.pig.backend.hadoop.executionengine.util.MapRedUtil - Total input paths to process : 1
(WN,127601)
(AA,59842)
(DL,57668)
(UA,57367)
(US,40943)
2024-01-29 15:53:52,426 [main] INFO  org.apache.pig.Main - Pig script completed in 11 seconds and 635 milliseconds (11635 ms)


## 5.- Resuelve el ejercicio 3 con Pig Latin

Se espera el siguiente resultado:

![solución 5](./img/5.png)

In [75]:
%%writefile company_recover.pig

-- registro de la librería PiggyBank
REGISTER piggybank.jar

-- Lectura CSV

AIRPORTS = LOAD '$airports_file' USING
    org.apache.pig.piggybank.storage.CSVExcelStorage(',','NO_MULTILINE','UNIX','SKIP_INPUT_HEADER')
    AS (airportid:chararray, city:chararray, state:chararray, airportname:chararray);

FLIGHTS = LOAD '$flights_file' USING
    org.apache.pig.piggybank.storage.CSVExcelStorage(',','NO_MULTILINE','UNIX','SKIP_INPUT_HEADER')
    AS (dayofmonth:int, dayofweek:int, carrier:chararray,
           depairport:chararray, arrairportid:chararray, depdelay:int, arrdelay:int);

-- Seleccionamos los vuelos retrasados
vuelos_retrasados = FILTER FLIGHTS BY depdelay > 15;

-- Seleccionamos los vuelos retrasados que llegaron sin retraso
vuelos_recuperados = FILTER vuelos_retrasados BY arrdelay <= 15;

-- Contamos los vuelos recuperados por cada compañía
contar_recuperados = GROUP vuelos_recuperados BY carrier;
total_recuperados = FOREACH contar_recuperados GENERATE group AS carrier, COUNT(vuelos_recuperados) AS vuelos_recuperados;

-- Contamos los vuelos retrasados por cada compañía
contar_retrasos = GROUP vuelos_retrasados BY carrier;
total_retrasos = FOREACH contar_retrasos GENERATE group AS carrier, COUNT(vuelos_retrasados) AS total_retrasos;

-- Calcular porcentaje de recuperación
recover = JOIN total_recuperados BY carrier, total_retrasos BY carrier;
recover = FOREACH recover GENERATE
    $0 AS carrier,
    ($1 * 1.0 / $3) AS recover;

-- Ordenar y limitar a 5 compañías
top5 = ORDER recover BY recover DESC;
top5 = LIMIT top5 5;

-- DUMP del TOP5
DUMP top5;


Overwriting company_recover.pig


In [76]:
! pig -x local -f company_recover.pig -param airports_file='airports.csv' -param flights_file='flights.csv' -param output_dir='pig/output/flights'

2024-01-29 17:07:59,858 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2024-01-29 17:07:59,858 INFO pig.ExecTypeProvider: Picked LOCAL as the ExecType
2024-01-29 17:07:59,895 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2024-01-29 17:07:59,895 [main] INFO  org.apache.pig.Main - Logging error messages to: /media/notebooks/pig_1706544479893.log
2024-01-29 17:07:59,904 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2024-01-29 17:08:00,009 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2024-01-29 17:08:00,057 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2024-01-29 17:08:00,058 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2024-01-29 17:

2024-01-29 17:08:01,229 [JobControl] INFO  org.apache.hadoop.mapreduce.Job - The url to track the job: http://localhost:8080/
2024-01-29 17:08:01,229 [Thread-6] INFO  org.apache.hadoop.mapred.LocalJobRunner - OutputCommitter set in config null
2024-01-29 17:08:01,245 [Thread-6] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
2024-01-29 17:08:01,245 [Thread-6] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2024-01-29 17:08:01,245 [Thread-6] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.reduce.markreset.buffer.percent
2024-01-29 17:08:01,247 [Thread-6] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - File Output Committer Algorithm version is 2
2024-01-29 17:08:01,247 [Thread-6] INFO  org.apache.hadoop.mapreduce.lib.o

2024-01-29 17:08:05,732 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - File Output Committer Algorithm version is 2
2024-01-29 17:08:05,732 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
2024-01-29 17:08:05,732 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.Task -  Using ResourceCalculatorProcessTree : [ ]
2024-01-29 17:08:05,733 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Processing split: Number of splits :1
Total Length = 33554432
Input split[0]:
   Length = 33554432
   ClassName: org.apache.hadoop.mapreduce.lib.input.FileSplit
   Locations:

-----------------------

2024-01-29 17:08:05,736 [LocalJobRunner Map Task Executor #0] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.PigRecor

2024-01-29 17:08:10,405 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.LocalJobRunner - 
2024-01-29 17:08:10,405 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Starting flush of map output
2024-01-29 17:08:10,405 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Spilling map output
2024-01-29 17:08:10,405 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - bufstart = 0; bufend = 288340; bufvoid = 104857600
2024-01-29 17:08:10,405 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - kvstart = 26214396(104857584); kvend = 26099064(104396256); length = 115333/6553600
2024-01-29 17:08:10,433 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Finished spill 0
2024-01-29 17:08:10,434 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.Task - Task:attempt_local533391016_0001_m_000002_0 is done. And is in the proces

2024-01-29 17:08:10,512 [pool-4-thread-1] INFO  org.apache.hadoop.mapred.Task - Task:attempt_local533391016_0001_r_000000_0 is done. And is in the process of committing
2024-01-29 17:08:10,516 [pool-4-thread-1] INFO  org.apache.hadoop.mapred.LocalJobRunner - 3 / 3 copied.
2024-01-29 17:08:10,516 [pool-4-thread-1] INFO  org.apache.hadoop.mapred.Task - Task attempt_local533391016_0001_r_000000_0 is allowed to commit now
2024-01-29 17:08:10,518 [pool-4-thread-1] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - Saved output of task 'attempt_local533391016_0001_r_000000_0' to file:/tmp/temp-679547720/tmp-1207363180
2024-01-29 17:08:10,520 [pool-4-thread-1] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - Saved output of task 'attempt_local533391016_0001_r_000000_0' to file:/tmp/temp-679547720/tmp-1789324695
2024-01-29 17:08:10,521 [pool-4-thread-1] INFO  org.apache.hadoop.mapred.LocalJobRunner - reduce > reduce
2024-01-29 17:08:10,521 [pool-4-thread-1] IN

...

2024-01-29 17:08:10,768 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.Task - Task:attempt_local346672689_0002_r_000000_0 is done. And is in the process of committing
2024-01-29 17:08:10,770 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.LocalJobRunner - 2 / 2 copied.
2024-01-29 17:08:10,770 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.Task - Task attempt_local346672689_0002_r_000000_0 is allowed to commit now
2024-01-29 17:08:10,771 [pool-9-thread-1] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - Saved output of task 'attempt_local346672689_0002_r_000000_0' to file:/tmp/temp-679547720/tmp1519621592
2024-01-29 17:08:10,771 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.LocalJobRunner - reduce > reduce
2024-01-29 17:08:10,772 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.Task - Task 'attempt_local346672689_0002_r_000000_0' done.
2024-01-29 17:08:10,772 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.Task - Final Counters for attempt_local346672689_0002_

2024-01-29 17:08:11,023 [pool-12-thread-1] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - File Output Committer Algorithm version is 2
2024-01-29 17:08:11,023 [pool-12-thread-1] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
2024-01-29 17:08:11,025 [pool-12-thread-1] INFO  org.apache.hadoop.mapred.Task -  Using ResourceCalculatorProcessTree : [ ]
2024-01-29 17:08:11,025 [pool-12-thread-1] INFO  org.apache.hadoop.mapred.ReduceTask - Using ShuffleConsumerPlugin: org.apache.hadoop.mapreduce.task.reduce.Shuffle@428312a
2024-01-29 17:08:11,025 [pool-12-thread-1] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-01-29 17:08:11,026 [pool-12-thread-1] INFO  org.apache.hadoop.mapreduce.task.reduce.MergeManagerImpl - MergerManager: memoryLimit=652528832, maxSingleShuffleLimit=163132208,

2024-01-29 17:08:11,252 [JobControl] INFO  org.apache.hadoop.mapreduce.Job - The url to track the job: http://localhost:8080/
2024-01-29 17:08:11,252 [Thread-32] INFO  org.apache.hadoop.mapred.LocalJobRunner - OutputCommitter set in config null
2024-01-29 17:08:11,256 [Thread-32] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
2024-01-29 17:08:11,256 [Thread-32] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2024-01-29 17:08:11,256 [Thread-32] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.reduce.markreset.buffer.percent
2024-01-29 17:08:11,256 [Thread-32] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - File Output Committer Algorithm version is 2
2024-01-29 17:08:11,256 [Thread-32] INFO  org.apache.hadoop.mapreduce

2024-01-29 17:08:11,453 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - HadoopJobId: job_local856459684_0004
2024-01-29 17:08:11,453 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - Processing aliases top5
2024-01-29 17:08:11,453 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - detailed locations: M: top5[37,7] C:  R: 
2024-01-29 17:08:11,454 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - 80% complete
2024-01-29 17:08:11,455 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-01-29 17:08:11,456 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-01-29 17:08:11,457 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024

2024-01-29 17:08:11,730 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - HadoopJobId: job_local389593575_0005
2024-01-29 17:08:11,730 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - Processing aliases top5
2024-01-29 17:08:11,730 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - detailed locations: M: top5[37,7] C:  R: 
2024-01-29 17:08:11,732 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-01-29 17:08:11,733 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-01-29 17:08:11,734 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-01-29 17:08:11,737 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - 100% complete
202